In [2]:
# This notebook uses a nomogram available on the web to create an additional feature for patients
# The nomogram is available at https://nomograms.mskcc.org/Breast/BreastSLNodeMetastasisPage.aspx
# The filling and submitting of the webform is automated using the 'pyautogui' module
from pyautogui import * 
import time
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', True)

nomogram_url = "https://nomograms.mskcc.org/Breast/BreastSLNodeMetastasisPage.aspx"
raw_data_path = "data/input/AllTranTrainVal.csv"

ModuleNotFoundError: No module named 'pyautogui'

In [ ]:
# Load raw data
df = pd.read_csv(raw_data_path)
df.head()

: 